In [1]:
from sentence_transformers import SentenceTransformer, util
import pandas as pd
import torch

c:\Users\Sahaana Venkat\OneDrive\Desktop\Worshop_march_2024\workshop\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data_for_recommendation = [
    {"text": "how to maintain health", "label": 0},
    {"text": "how to protect health", "label": 1},
    {"text": "factors affecting health", "label": 2},
    {"text": "bad factors for health", "label": 3},
    {"text": "good factors for health", "label": 4},
    {"text": "preventing shutdown in health", "label": 5},
    {"text": "optimizing health performance", "label": 6},
    {"text": "extending lifespan in health", "label": 7},
    {"text": "enhancing health efficiency", "label": 8},
    {"text": "key parameters to monitor in health", "label": 9}]

In [3]:
### Conceptual similarity

In [4]:
# Load a pre-trained BERT model
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
df = pd.DataFrame(data_for_recommendation)

# Embed the texts
embeddings = model.encode(df['text'], convert_to_tensor=True)

modules.json: 100%|██████████| 229/229 [00:00<00:00, 212kB/s]
c:\Users\Sahaana Venkat\OneDrive\Desktop\Worshop_march_2024\workshop\Lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Sahaana Venkat\.cache\huggingface\hub\models--sentence-transformers--paraphrase-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  wa

In [5]:
# Function to calculate similarity
def calculate_similarity(query, embeddings):
    query_embedding = model.encode(query, convert_to_tensor=True)
    cos_scores = util.pytorch_cos_sim(query_embedding, embeddings)[0]
    return cos_scores

In [6]:
# Example: Find similar texts for a given query
def input_text_model(input_text):
    """_summary_

    Args:
        input_text (_type_): _description_

    Returns:
        _type_: _description_
    """
    
    input_sentence_processed = ''.join([i for i in input_text if not i.isdigit()])
    query = input_sentence_processed
    query_similarity = calculate_similarity(query, embeddings)
    top_n = 10
# Convert PyTorch tensor to a NumPy array and move to CPU if necessary
    top_indices = query_similarity.argsort().cpu().numpy()[-top_n:]
    similar_texts = df.loc[top_indices, 'label'].tolist()
    label=similar_texts[-1]
    return label

In [7]:
label=input_text_model("Tips for preserving health?")
print(label)

0


In [8]:
#### Cosine similarity

In [10]:
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.metrics.pairwise import cosine_similarity
import torch
import numpy as np


# Load BERT model and tokenizer
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Tokenize and encode the dataset for recommendation function only
tokenized_data_recommendation = tokenizer(
    [item["text"] for item in data_for_recommendation],
    padding=True,
    truncation=True,
    return_tensors="pt"
)

def input_text_model(input_text):
    """Predict label for input text"""
    # Process input text
      
    # Tokenize the input sentence
    tokenized_input = tokenizer(
        input_text,
        padding=True,
        truncation=True,
        return_tensors="pt"
    )
    
    # Get the embeddings for the input sentence
    with torch.no_grad():
        model_output = model(**tokenized_input)
        input_embeddings = model_output.logits.detach().numpy()
    
    # Get the embeddings for sentences in both datasets
    embeddings_recommendation = model(**tokenized_data_recommendation).logits.detach().numpy()

    # Calculate cosine similarity
    similarity_recommendation = cosine_similarity(input_embeddings, embeddings_recommendation)

    # Determine the label based on higher cosine similarity
    most_similar_index_recommendation = np.argmax(similarity_recommendation)
    label_recommendation = data_for_recommendation[most_similar_index_recommendation]["label"]
    return label_recommendation

# Sample usage

label = input_text_model("Tips for preserving health?")
print(label)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


6
